# Biodivine AEON

This notebook illustrates how to use Python bindings of the tool AEON to symbolically analyze the attractors of non-trivial Boolean networks.

No special knowledge is necessary to understand the contents of this particular notebook. However, to successfully apply AEON to other problems, some basic understanding of AEON's notion of *partially specified Boolean network* and the underlying symbolic representation using BDDs is recommended (these are explained [here](./bn_tutorial.ipynb) and [here](bdd_tutorial.ipynb)).

We start our exploration by loading a Boolean network from CellCollective:

In [1]:
from biodivine_aeon import *
from pathlib import Path
import cellcollective

# Load the network from CellCellective.
sbml = cellcollective.load("https://research.cellcollective.org/#module/36604:1/signaling-pathway-for-butanol-production-in-clostridium-beijerinckii-nrrl-b598/10")

# Open the loaded SBML and parse it as a Boolean network.
model = BooleanNetwork.from_sbml(Path(sbml.localfile).read_text())
print(f"Loaded model with {model.num_vars()} variables.")

This notebook has been executed using the docker image `colomoto/colomoto-docker` built on `2021-11-16`

/notebook/tutorials/AEON/cellcollective-36604-1.sbml

Loaded model with 66 variables.


Right now, the model has no explicit paramters, but there are several variables for which no update function is given in the SBML file. AEON will automatically consider these as implicit unknown parameters of the network:

In [2]:
print(f"Number of explicit parameters: {model.num_parameters()}.")
implicit_parameters = 0
for var in model.variables():
    if model.get_update_function(var) == None:
        implicit_parameters += 1
print(f"Number of implicit parameters: {implicit_parameters}.")

Number of explicit parameters: 0.
Number of implicit parameters: 13.


Before we can explore the attractors of this network, first let us observe that the network as downloaded from CellCollective is actually not logically consistent with its regulatory graph. AEON will notify us about this fact when we try to construct the network's asynchronous state-transition graph:

In [3]:
try:
    stg = SymbolicAsyncGraph(model)
except Exception as e:
    print(e)

No update functions satisfy given constraints: 
 - spoIIE not activating in spoIIAB.
 - sporulation has no effect in glucose___PTS.
 - sigA has no effect in spo0A_p.



Fortunately, AEON can tell us exactly which properties are violated. So if we want to fix these issues, we know what variables to focus on. For now, we simply relax the conditions in the regulatory graph such that the existing update functions are valid:

In [4]:
# Export the Boolean network as `.aeon` model string.
model_aeon = model.to_aeon()

# Relax the problematic regulation constraints:

# Mark regulation "spoIIE -> spoIIAB" as non-monotonous:
model_aeon = model_aeon.replace("spoIIE -> spoIIAB", "spoIIE -? spoIIAB")
# Mark regulation "sporulation -| glucose___PTS" as non-essential:
model_aeon = model_aeon.replace("sporulation -| glucose___PTS", "sporulation -|? glucose___PTS")
# Mark regulation "sigA -> spo0A_p" as non-essential:
model_aeon = model_aeon.replace("sigA -> spo0A_p", "sigA ->? spo0A_p")

# Load back the modified model:
model = BooleanNetwork.from_aeon(model_aeon)

Now we can actually compute the attractors quite easily. However, note that since AEON has to consider each possible variant of the network, the computation can still take several seconds or minutes (recall that we actually have `2^13 = 8192` possible networks).

In [5]:
stg = SymbolicAsyncGraph(model)
attractors = find_attractors(stg)
attractors

[ColoredVertexSet(cardinality = 867718363543552, unique vertices = 867718363543552, unique colors = 6704),
 ColoredVertexSet(cardinality = 15959465984, unique vertices = 15959465984, unique colors = 1488)]

As we can see, AEON found two separate attractor sets. Since the sets are very large, only a summary of each set is printed. Each set has millions of vertices and thousands of colors -- AEON uses the term "color" to refer to concrete models arising for different parametrisations.

However, in this case, we can actually verify that these two sets are color-disjoint. This means that each possible network actually still only has one attractor. Consequently, we can just merge the two sets and treat them as a single attractor:

In [6]:
colors_in_both_sets = attractors[0].colors().intersect(attractors[1].colors())
print(f"Number of colors appearing in both sets: {colors_in_both_sets.cardinality()}")

attractor = attractors[0].union(attractors[1])
attractor

Number of colors appearing in both sets: 0.0


ColoredVertexSet(cardinality = 867734323009536, unique vertices = 867734323009536, unique colors = 8192)

And just as we suspected, there is no intersection between the two sets. This can happen due to how the underlying algorithm works: some stable states are detected first, and more complex attractors are detected afterwards. However, since there are no intersections between these two sets, we can unify them and consider them as one attractor. 

Now, to better understand what is happening in this attractor, we can classify the colors based on their long-term behavioural characteristics (this can also take a few second to compute):

In [7]:
classes = classify_attractor(stg, attractor)
classes

{'stability': ColorSet(cardinality=2048),
 'disorder': ColorSet(cardinality=6144)}

In this case, we can see that 2048 colors correspond to a stable state, and 6144 colors correspond to a disordered attractor (i.e. an attractor that is neither a stable state nor a cycle). In this case, we have no oscillating (cyclic) attractors. 

We can now use these classes to divide the `attractor` set based on its behaviour:

In [8]:
stable_attractor = attractor.intersect_colors(classes["stability"])
disordered_attractor = attractor.intersect_colors(classes["disorder"])

print(stable_attractor)
print(disordered_attractor)

ColoredVertexSet(cardinality = 2048, unique vertices = 2048, unique colors = 2048)
ColoredVertexSet(cardinality = 867734323007488, unique vertices = 867734323007488, unique colors = 6144)


To obtain some useful information from these sets, we can for example look at the possible values of specific network variables within each set. In this model, there is a variable called `sporulation` that is of particular interest, since it signifias that the cell is "hybernating". 

To study whether sporulation is on or off in the possible attractors, we can simply construct the set of all states where sporulation is on, and then compare it to the attractor sets:

In [9]:
sporulation_on = stg.fix_variable("sporulation", True)

on_in_stable_attractor = stable_attractor.intersect(sporulation_on).vertices().cardinality()
off_in_stable_attractor = stable_attractor.minus(sporulation_on).vertices().cardinality()

print("Sporulation is ON in", on_in_stable_attractor, "stable states.")
print("Sporulation is OFF in", off_in_stable_attractor, "stable states.")
print("Sporulation is ON in", round((on_in_stable_attractor / (on_in_stable_attractor + off_in_stable_attractor)) * 100.0, 2), "% of stable attractor states.")

on_in_disorder_attractor = disordered_attractor.intersect(sporulation_on).vertices().cardinality()
off_in_disorder_attractor = disordered_attractor.minus(sporulation_on).vertices().cardinality()

print("Sporulation is ON in", on_in_disorder_attractor, "disorder states.")
print("Sporulation is OFF in", off_in_disorder_attractor, "disorder states.")
print("Sporulation is ON in", round((on_in_disorder_attractor / (on_in_disorder_attractor + off_in_disorder_attractor)) * 100.0, 2), "% of disordered attractor states.")

Sporulation is ON in 2048.0 stable states.
Sporulation is OFF in 0.0 stable states.
Sporulation is ON in 100.0 % of stable attractor states.
Sporulation is ON in 335223420223488.0 disorder states.
Sporulation is OFF in 532510902784000.0 disorder states.
Sporulation is ON in 38.63 % of disordered attractor states.


Interestingly, we have discovered that in every stable state, `sporulation` is active. We can thus use this fact to pick a witness network which guarantees that eventually, `sporulation` will always be active in a stable attractor. 

This witness network is completely specified (i.e. has no parameters or uninterpreted functions). In other words, all of the explicit and implicit parameters are fixed to guarantee the desired behaviour:

In [10]:
witness = stg.pick_witness(classes["stability"])

print(f"Number of explicit parameters: {witness.num_parameters()}.")
implicit_parameters = 0
for var in witness.variables():
    if witness.get_update_function(var) == None:
        implicit_parameters += 1
print(f"Number of implicit parameters: {implicit_parameters}.")

Number of explicit parameters: 0.
Number of implicit parameters: 0.


Finally, since computing the attractors can take a long time, it may be necessary to save them into a file for further processing. For this, we can export the underlying BDD representation:

In [11]:
Path("stable_attractor.bdd").write_text(stable_attractor.to_bdd().to_raw_string())
Path("disordered_attractor.bdd").write_text(disordered_attractor.to_bdd().to_raw_string())

62415

With the BDDs saved, we can always reload them from disk. However, remeber that the BDD does not contain any information about the Boolean network from which it was constructed! So you must make sure that you only import BDDs that were created using the same Boolean model. Ideally, always save an `.aeon` or `.sbml` file along with any raw BDD data.

In [12]:
stable_reloaded = stg.empty_colored_vertices().copy_with_raw_string(Path("stable_attractor.bdd").read_text())
disordered_reloaded = stg.empty_colored_vertices().copy_with_raw_string(Path("disordered_attractor.bdd").read_text())

print(stable_reloaded)
print(disordered_reloaded)

ColoredVertexSet(cardinality = 2048, unique vertices = 2048, unique colors = 2048)
ColoredVertexSet(cardinality = 867734323007488, unique vertices = 867734323007488, unique colors = 6144)


As we can see, the sets are the same as the ones that we saved into the files.

Finally, let us quickly note that using AEON, you can also compute reachability from/to a specific symbolic set. Furthermore, you can specify a subset within which the reachability should be computed. Using this mechanism, you can often build more complex analysis techniques. For example, you can analyse the strong and weak basin of a particular attractor, or check for reachable attractors from a specific initial state.

In [13]:
# Pick a set containing a single arbitrary vertex from the whole state space:
vertex = stg.unit_colored_vertices().pick_vertex()

# The actual vertex is just a list of Boolean values.
print("Vertex", vertex.vertices().list_vertices()[0])

# Compute the set of states backward-reachable from the vertex.
basin = reach_bwd(stg, vertex)
print("Basin", basin)

# Compute the strongly-connected-component in which the vertex resides.
scc = reach_fwd(stg, vertex, basin)
print("SCC", scc)

Vertex [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Basin ColoredVertexSet(cardinality = 11558771119839542000000, unique vertices = 63365787494591230000, unique colors = 8192)
SCC ColoredVertexSet(cardinality = 8192, unique vertices = 1, unique colors = 8192)


For example, for an initial state where all variables are inactive, we can see that its (weak) basin is quite large, but the vertex is in fact a trivial SCC (the whole strongly connected component consists only of one vertex, regardless of color). 


So far, this is the end of our demo. In case of any issues, feel free to contact us on [github](https://github.com/sybila/biodivine-aeon-py)! 

To learn more about what features and functions are available in AEON.py, you can follow the tuturial on [partially specified Boolean networks](./bn_tutorial.ipynb) and [symbolic computation using BDDs](bdd_tutorial.ipynb).

Alternatively, you can also explore other case studies using AEON, which often include more complex examples:
 
 * [T-cell survival signaling](https://deepnote.com/@daemontus/Dynamical-Analysis-of-a-T-cell-Survival-Network-5hoh-QdyRMOM-wV6RLyJyw)
 * [Butanol production pathway](https://deepnote.com/@daemontus/Function-Repair-in-Butanol-Production-Pathway-kkkfVNKuSaKUWTabJkBYgQ)
 * [Type-1 interferon pathway](https://deepnote.com/@daemontus/Drug-Induced-Effects-on-Interferon-Signalling-ynwY09IvQeOWH3ey24uxew)